In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
from LibraryCode.DataReader import DataReader
from LibraryCode.RNN import RNN
from LibraryCode.vocab import *
import random

**1.Generate the Dictionary**

In [ ]:
gen_data_and_vocab()

**2.Encoding Data**

In [ ]:
dir_data = "../Session 4/datasets/"
train_path = os.path.join(dir_data, "w2v", "20news_train_raw.txt")
test_path = os.path.join(dir_data, "w2v", "20news_test_raw.txt")
vocab_path = os.path.join(dir_data, "w2v", "vocab-raw.txt")

In [ ]:
encode_data(train_path,vocab_path)
encode_data(test_path,vocab_path)

**3.RNN**

In [ ]:
train_encoded_path = os.path.join(dir_data, "w2v", "20news-train-encoded.txt")
test_encoded_path = os.path.join(dir_data, "w2v", "20news-test-encoded.txt")

In [ ]:
loss_report = []
accuracy_report = []

In [ ]:
def trainAndEvaluateRNN():
  with open(vocab_path) as f:
    vocab_size = len(f.read().splitlines())

  random.seed(2022)
  rnn = RNN(
      vocab_size=vocab_size,
      embedding_size=300,
      lstm_size=50,
      batch_size=50
  )
  predicted_labels, loss = rnn.build_graph()
  train_op = rnn.trainer(loss=loss, learning_rate=0.01)

  with tf.Session() as sess:
    train_data_reader = DataReader(
        data_path=train_encoded_path,
        batch_size=50
    )
    test_data_reader = DataReader(
        data_path=test_encoded_path,
        batch_size=50
    )
    step = 0
    MAX_STEP = 10000

    sess.run(tf.global_variables_intializer())
    while step < MAX_STEP:
      next_train_batch = train_data_reader.next_batch()
      train_data, train_labels, train_sentence_lengths, train_final_tokens = next_train_batch
      plabels_eval, loss_eval, _ = sess.run(
          [predicted_labels, loss, train_op],
          feed_dict={
              rnn._data: train_data,
              rnn._labels: train_labels,
              rnn._sentence_lengths: train_sentence_lengths,
              rnn._final_tokens: train_final_tokens
          }
      )
      step += 1
      if step % 20 == 0:
        loss_report.append(loss_eval)
        print ('loss:', loss_eval)
      if train_data_reader._batch_id == 0:
        num_true_preds = 0
        while True:
          next_test_batch = test_data_reader.next_batch()
          test_data, test_labels, test_sentence_lengths, test_final_tokens = next_test_batch

          test_plabels_eval = sess.run(
              predicted_labels,
              feed_dict={
                  rnn._data: test_data,
                  rnn._labels: test_labels,
                  rnn._sentence_lengths: test_sentence_lengths,
                  rnn._final_tokens: test_final_tokens
              }
          )
          matches = np.equal(test_plabels_eval, test_labels)
          num_true_preds += np.sum(matches.astype(float))

          if test_data_reader._batch_id == 0:
            break

        accuracy_report.append(num_true_preds * 100. / test_data_reader._size)
        print ('Epoch:', train_data_reader._num_epoch)
        print ('Accuracy on test data:', num_true_preds * 100. / len(test_data_reader._data))

In [ ]:
trainAndEvaluateRNN()